In [1]:
import pandas as pd
from talib import abstract
import numpy as np

In [2]:
stock = "RMD"

df = pd.read_csv(f"15YearsDatasets/{stock}.csv", parse_dates=True, index_col=0)
df.columns = ["open", "high", "low", "close", "adj close", "volume"]
df.head()

,open,high,low,close,adj close,volume
Date,,,,,,
2005-01-03,12.8000,12.9350,12.6825,12.7025,11.016774,955600
2005-01-04,12.7125,12.7250,12.5825,12.6325,10.956066,1304000
2005-01-05,12.6275,12.7250,12.5950,12.6500,10.971242,776800
2005-01-06,12.7125,12.7625,12.6475,12.6475,10.969073,594400
2005-01-07,12.6375,12.6500,12.5625,12.5975,10.925709,456800


In [3]:
df.interpolate(inplace=True)

In [4]:
df["SMA"] = abstract.SMA(df, timeperiod=7)
df["WMA"] = abstract.WMA(df, timeperiod=7)

In [5]:
df["RSI"] = abstract.RSI(df, timeperiod=7)

In [6]:
df["CCI"] = abstract.CCI(df, timeperiod=10)
df["AD"] = abstract.AD(df, timeperiod=10)

In [7]:
df["STOCHRSIk"] = abstract.STOCHRSI(df, timeperiod=7)["fastk"]
df["STOCHRSId"] = abstract.STOCHRSI(df, timeperiod=7)["fastd"]
df["WILLR"] = abstract.WILLR(df, timeperiod=7)

In [8]:
df["MACD"] = abstract.MACDFIX(df)["macd"]
df["MACDsignal"] = abstract.MACDFIX(df)["macdsignal"]

In [9]:
df = df.iloc[33:]

# ABOVE USED IN PAPER 

In [10]:
#df["ROC4"] = abstract.ROCP(df, timeperiod=4) * 100

In [11]:
#df["TRIX"] = abstract.TRIX(df)

In [12]:
# Select trend from SMA14 and SMA 50
def MA_trend(ma, price):
    return 1 if price > ma else -1

df["SMA_trend"] = df.apply(lambda x: MA_trend(x["SMA"], x["close"]), axis=1)
df["WMA_trend"] = df.apply(lambda x: MA_trend(x["WMA"], x["close"]), axis=1)

<ipython-input-12-b25718390c1c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SMA_trend"] = df.apply(lambda x: MA_trend(x["SMA"], x["close"]), axis=1)
<ipython-input-12-b25718390c1c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["WMA_trend"] = df.apply(lambda x: MA_trend(x["WMA"], x["close"]), axis=1)


In [13]:
# Select trend from RSI
RSI_trend = [0] * len(df)
for i in range(len(df)):
    if i == 0:
        continue
    lastrsi = df.loc[df.index[i-1], "RSI"]
    rsi = df.loc[df.index[i], "RSI"]
    if rsi > 70:
        RSI_trend[i] = -1
    elif rsi < 30:
        RSI_trend[i] = 1
    elif rsi > lastrsi:
        RSI_trend[i] = 1
    else:
        RSI_trend[i] = -1


df["RSI_trend"] = RSI_trend

<ipython-input-13-bd738f0b9da3>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RSI_trend"] = RSI_trend


In [14]:
# Select buy trend from STOCHRSI
STOCHRSIk_trend = [0] * len(df)
STOCHRSId_trend = [0] * len(df)
WILLR_trend = [0] * len(df)
CCI_trend = [0] * len(df)
AD_trend = [0] * len(df)
for i in range(len(df)) :
    if i == 0:
        continue
    lastk = df.loc[df.index[i-1], "STOCHRSIk"]
    lastd = df.loc[df.index[i-1], "STOCHRSId"]
    lastwillr = df.loc[df.index[i-1], "WILLR"]
    lastcci = df.loc[df.index[i-1], "CCI"]
    lastad = df.loc[df.index[i-1], "AD"]
    
    k = df.loc[df.index[i], "STOCHRSIk"]
    d = df.loc[df.index[i], "STOCHRSId"]
    willr = df.loc[df.index[i], "WILLR"]
    cci = df.loc[df.index[i], "CCI"]
    ad = df.loc[df.index[i], "AD"]
    # identify trends
    # K
    if k > lastk:
        STOCHRSIk_trend[i] = 1
    else:
        STOCHRSIk_trend[i] = -1
    # D
    if d > lastd:
        STOCHRSId_trend = 1
    else:
        STOCHRSId_trend = -1
    # WILLR
    if willr > lastwillr:
        WILLR_trend = 1
    else:
        WILLR_trend = -1
    # CCI
    if cci > 150:
        CCI_trend = -1
    elif cci < -150:
        CCI_trend = 1
    elif cci > lastcci:
        CCI_trend = 1
    else:
        CCI_trend = -1
    # WILLR
    if ad > lastad:
        AD_trend = 1
    else:
        AD_trend = -1
        
        
df["STOCHRSIk_trend"] = STOCHRSIk_trend
df["STOCHRSId_trend"] = STOCHRSId_trend
df["WILLR_trend"] = WILLR_trend
df["CCI_trend"] = CCI_trend
df["AD_trend"] = AD_trend

<ipython-input-14-a364b23c8fc3>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["STOCHRSIk_trend"] = STOCHRSIk_trend
<ipython-input-14-a364b23c8fc3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["STOCHRSId_trend"] = STOCHRSId_trend
<ipython-input-14-a364b23c8fc3>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [15]:
# Select trend from MACD
def MACD_trend(macd, signal):
    if macd > signal:
        return 1
    else:
        return -1

df["MACD_trend"] = df.apply(lambda x: MACD_trend(x["MACD"], x["MACDsignal"]), axis=1)

<ipython-input-15-da4b33cb6bde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MACD_trend"] = df.apply(lambda x: MACD_trend(x["MACD"], x["MACDsignal"]), axis=1)


In [16]:
# Select trend from TRIX
def TRIX_trend(trix):
    if trix > 0:
        return 1
    else:
        return -1

#df["TRIX_trend"] = df.apply(lambda x: TRIX_trend(x["TRIX"]), axis=1)

In [17]:
df.columns

Index(['open', 'high', 'low', 'close', 'adj close', 'volume', 'SMA', 'WMA',
       'RSI', 'CCI', 'AD', 'STOCHRSIk', 'STOCHRSId', 'WILLR', 'MACD',
       'MACDsignal', 'SMA_trend', 'WMA_trend', 'RSI_trend', 'STOCHRSIk_trend',
       'STOCHRSId_trend', 'WILLR_trend', 'CCI_trend', 'AD_trend',
       'MACD_trend'],
      dtype='object')

In [18]:
df = df.iloc[1:]
outputdf = df.drop(["open", "high", "low", "volume", "adj close",
         'SMA', 'WMA', 'RSI', 'CCI', 'AD',
         'STOCHRSIk', 'STOCHRSId',
         'WILLR', 'MACD', 'MACDsignal'], axis=1)

In [19]:
outputdf.isnull().values.any()

False

In [20]:
len(outputdf)

3741

In [21]:
outputdf.head()

,close,SMA_trend,WMA_trend,RSI_trend,STOCHRSIk_trend,STOCHRSId_trend,WILLR_trend,CCI_trend,AD_trend,MACD_trend
Date,,,,,,,,,,
2005-02-22,14.7750,-1,-1,-1,-1,1,1,1,1,1
2005-02-23,14.7600,-1,-1,-1,-1,1,1,1,1,-1
2005-02-24,14.7550,-1,-1,-1,-1,1,1,1,1,-1
2005-02-25,14.7950,-1,-1,1,1,1,1,1,1,-1
2005-02-28,14.7375,-1,-1,-1,-1,1,1,1,1,-1


In [22]:
from pathlib import Path
path = Path(f"15YearsDatasets/{stock}/")
path.mkdir(parents=True, exist_ok=True)
outputdf.to_csv(f"{path}/{stock}trends2.csv", index=False)

In [23]:
# Use 70% for training, and the rest for validation
perc70 = int(len(outputdf) * 0.7)
training_df = outputdf.iloc[:perc70]
validation_df = outputdf.iloc[perc70:]

In [24]:
training_df.to_csv(f"{path}/{stock}training.csv", index=False)
validation_df.to_csv(f"{path}/{stock}validation.csv", index=False)